<center><img src="https://emojipedia-us.s3.dualstack.us-west-1.amazonaws.com/thumbs/240/apple/285/woman-artist_1f469-200d-1f3a8.png" width="120">
</center>

### <center>Use this notebook to run your DALL-E server</center>
### <center> [DALL-E Playground Repository](https://github.com/saharmor/dalle-playground) </center>

#####<center>This notebook is based on the amazing [DALL·E Mini](https://github.com/borisdayma/dalle-mini) by [Boris Dayma](https://github.com/borisdayma) et al.</center>

## Install dependencies

In [ ]:
!git clone https://github.com/saharmor/dalle-playground.git
!pip install -r dalle-playground/backend/requirements.txt
!npm install -g localtunnel

## Choose a model

* **DALL-E Mini**
  * The original DALL-E Mini. Fastest yet suboptimal results
  * Runs well on a 4GB GPU and the Google Colab free tier
* **DALL-E Mega**
  * The advanced version of DALL-E Mini. Requires more compute and VRAM
  * Runs well on a 8GB GPU and recommended on the Google Colab Pro tier
* **DALL-E Mega Full**
  * The most performant DALL-E version. Requires significantly more computre and VRAM
  * Runs well on a 12GB GPU and recommended on the Google Colab Pro+ tier


In [ ]:
#@title Select a model { display-mode: "form" }

dalle_model = 'Mini' #@param ["Mini", "Mega", "Mega_full"]

In [ ]:
from threading import Thread

def app():
  !python dalle-playground/backend/app.py 8000 $dalle_model

if __name__ == '__main__':
    t1 = Thread(target = app)
    a = t1.start()
    !lt --port 8000

### Now, take the url you got from localtunnel (should look like `your url is: https://xxxxxx.loca.lt`), replace it within the url here https://saharmor.github.io/dalle-playground/?backendUrl=https://xxxxxx.loca.lt and run it in the browser. 

### Let the fun begin ✨